**Связь docker контейнера и файловой системы хоста**

# Volume

Это файлы которые лежат на хосте и не зависят от конкретного образа, но зависят от docker в целом. **Основное предназначение** - хранение данных.

### Комманды `volume`

Любая комманда для работы с volume начитается с `docker volume`.

- `ls` - отобразить доступные volumes;
- `rm` - удалить volume;

### Тонкости

Некоторые контейнеры, когда поднимаются создают себе volume автоматически. Потому в один момент можно обнаружить, что у тебя весь диск забит. Например `yandex/clickhouse-server`. Так в следующем примере я поднимаю несколько контейнеров `yandex/clickhouse-server` и покзываю, что каждый из них создал по volume.

In [6]:
%%bash
# поднимаю clickhouse
docker run -d --name db_1 --rm yandex/clickhouse-server
docker run -d --name db_2 --rm yandex/clickhouse-server
docker run -d --name db_3 --rm yandex/clickhouse-server

# смотрю список volume -
docker volume ls
docker stop db_1 db_2 db_3

82d50d878c1babea30805e20f9aaedf0198fd2617207f2891528f190d210bbad
36ae71fb6eb78cc91e958039ab53f093807faa43c82915af6a76b4ef482da0ce
60dc2f71e2b29277b8eda6be533ab42547d0061b64f115b87a1052a1541bb7c9
DRIVER    VOLUME NAME
local     2a6676858530a9d28db91bcea872748ad2516ff712716293c3cd42864f79669d
local     994071ad6640596efb04194a5c396696269a1c7bbd6c3d61861899f5bac64396
local     ef1d5dfad191f8c46fff3c140b26b40d3e89f2adefbb66de44c17d9e4d36e5e3
db_1
db_2
db_3


# Bind mount

Можно примонтировать к контейнеру любую папку или файл в любом месте на хосте. **Основное предназначение** - входная информация по типу ключей.

Bind mount создается через использование опции `--v <полный путь к файлу/папке на хосте:полный путь к файлу папке в контейнере>` комманды `run`.

In [2]:
%%bash
# убежадемся, что изначально файл пустой
cat my_datafile.csv
# потом запускаем контейнер монтируя bindmount
docker run --rm -itd -v $(pwd)/my_datafile.csv:/experimental/my_datafile.csv --name bm_ex file_example
# запускаем программу, которая должна добавить строчку в файл
docker exec bm_ex python3 experimental.py
# проверяем файл НА ХОСТЕ
cat my_datafile.csv

docker stop bm_ex

col1,col2
9f09b068890c28544b1892a4ba050e128016703c847248ba0d96a0f8861ba506
col1,col2
0.48907447800000703,0.3088149216473689
bm_ex


# Команда `cp`

Поможет провести коппирование в работающий контейнер. Синтаксис использования комманды следующий:

`docker cp <путь на хосте> <контейнер в который производится копирование>:<путь в контейнере>`

Далее пример, как я в работающий контейнер закину файл "my_datafile".

In [1]:
%%bash
# поднимаю контейнер в фоновом режиме
docker run -itd --rm --name c1 file_example
echo ""
# показываю что в конейнере нету файла
docker exec c1 ls
echo ""
# копирую файл через cp
docker cp my_datafile.csv c1:experimental/my_datafile.csv
# сомтрю в папку теперь
docker exec c1 ls
docker stop c1

0a4764eb41d1a3675983044244de0a3d4f891309f3c436413db479839b21abb5

experimental.py

experimental.py
my_datafile.csv
c1
